In [1]:
import numpy as np
import torch
import torch.nn.functional
from e3nn import o3
from e3nn.util import jit
from scipy.spatial.transform import Rotation as R

from mace import data, modules, tools
from mace.tools import torch_geometric

torch.set_default_dtype(torch.float64)

config = data.Configuration(
    atomic_numbers=np.array([8, 4, 1]),
    positions=np.array(
        [
            [0.0, -2.0, 0.0],
            [1.0, 0.0, 0.0],
            [0.0, 1.0, 0.0],
        ]
    ),
    forces=np.array(
        [
            [0.0, -1.3, 0.0],
            [1.0, 0.2, 0.0],
            [0.0, 1.1, 0.3],
        ]
    ),
    energy=-1.5,
    charges=np.array([-2.0, 1.0, 1.0]),
    dipole=np.array([-1.5, 1.5, 2.0]),
)
# Created the rotated environment
rot = R.from_euler("z", 60, degrees=True).as_matrix()
positions_rotated = np.array(rot @ config.positions.T).T
config_rotated = data.Configuration(
    atomic_numbers=np.array([8, 4, 1]),
    positions=positions_rotated,
    forces=np.array(
        [
            [0.0, -1.3, 0.0],
            [1.0, 0.2, 0.0],
            [0.0, 1.1, 0.3],
        ]
    ),
    energy=-1.5,
    charges=np.array([-2.0, 1.0, 1.0]),
    dipole=np.array([-1.5, 1.5, 2.0]),
)
table = tools.AtomicNumberTable([1, 4, 8])
atomic_energies = np.array([1.0, 3.0, 4.0], dtype=float)



In [2]:

model_config = dict(
    r_max=5,
    num_bessel=8,
    num_polynomial_cutoff=6,
    max_ell=2,
    interaction_cls=modules.interaction_classes[
        "RealAgnosticResidualInteractionBlock"
    ],
    interaction_cls_first=modules.interaction_classes[
        "RealAgnosticResidualInteractionBlock"
    ],
    num_interactions=5,
    num_elements=3,
    hidden_irreps=o3.Irreps("32x0e + 32x1o"),
    MLP_irreps=o3.Irreps("16x0e"),
    gate=torch.nn.functional.silu,
    atomic_energies=atomic_energies,
    avg_num_neighbors=8,
    atomic_numbers=table.zs,
    correlation=3,
    radial_type="bessel",
)

model = modules.MACE(**model_config)
model_compiled = jit.compile(model)


atomic_data = data.AtomicData.from_config(config, z_table=table, cutoff=3.0)
atomic_data2 = data.AtomicData.from_config(
    config_rotated, z_table=table, cutoff=3.0
)

data_loader = torch_geometric.dataloader.DataLoader(
    dataset=[atomic_data, atomic_data2],
    batch_size=2,
    shuffle=True,
    drop_last=False,
)
batch = next(iter(data_loader))
output1 = model(batch.to_dict(), training=True)
output2 = model_compiled(batch.to_dict(), training=True)
assert torch.allclose(output1["energy"][0], output2["energy"][0])
assert torch.allclose(output2["energy"][0], output2["energy"][1])


/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warning

In [3]:
llpr_model = modules.models.LLPredRigidityMACE(model, ll_feat_format="avg_over_atom", **model_config)

/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warning

In [4]:
llpr_model(batch)

{'energy': tensor([7.4713, 7.4713], grad_fn=<SumBackward1>),
 'node_energy': tensor([3.6912, 2.9839, 0.7962, 3.6912, 2.9839, 0.7962],
        grad_fn=<SumBackward1>),
 'contributions': tensor([[ 8.0000e+00,  9.9800e-02, -6.0861e-01, -5.7277e-02,  4.1010e-02,
          -3.6009e-03],
         [ 8.0000e+00,  9.9800e-02, -6.0861e-01, -5.7277e-02,  4.1010e-02,
          -3.6009e-03]], grad_fn=<StackBackward0>),
 'forces': tensor([[-0.0524,  0.0790, -0.0000],
         [-0.0269, -0.0999, -0.0000],
         [ 0.0793,  0.0209, -0.0000],
         [ 0.0422,  0.0849, -0.0000],
         [-0.0999, -0.0266, -0.0000],
         [ 0.0577, -0.0583, -0.0000]]),
 'virials': None,
 'stress': None,
 'displacement': tensor([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
 'll_feats': tensor([[-0.0152,  0.0859, -0.0458,  ...,  0.0025,  0.0312,  0.0097],
         [-0.0152,  0.0859, -0.0458,  ...,  0.0025,  0.0312,  0.00

In [6]:
llpr_model = modules.models.LLPredRigidityMACE(model, ll_feat_format="sum_over_atom", **model_config)
llpr_model.compute_covariance(data_loader)

/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/sanggyu/miniconda3/envs/mace/lib/python3.12/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warning

In [16]:
llpr_model.compute_inv_covariance(0.1, 0.00031)

In [17]:
llpr_model(batch)

{'energy': tensor([7.4713, 7.4713], grad_fn=<SumBackward1>),
 'node_energy': tensor([3.6912, 2.9839, 0.7962, 3.6912, 2.9839, 0.7962],
        grad_fn=<SumBackward1>),
 'contributions': tensor([[ 8.0000e+00,  9.9800e-02, -6.0861e-01, -5.7277e-02,  4.1010e-02,
          -3.6009e-03],
         [ 8.0000e+00,  9.9800e-02, -6.0861e-01, -5.7277e-02,  4.1010e-02,
          -3.6009e-03]], grad_fn=<StackBackward0>),
 'forces': tensor([[-0.0524,  0.0790, -0.0000],
         [-0.0269, -0.0999, -0.0000],
         [ 0.0793,  0.0209, -0.0000],
         [ 0.0422,  0.0849, -0.0000],
         [-0.0999, -0.0266, -0.0000],
         [ 0.0577, -0.0583, -0.0000]]),
 'virials': None,
 'stress': None,
 'displacement': tensor([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
 'll_feats': tensor([[-0.0456,  0.2577, -0.1373,  ...,  0.0076,  0.0935,  0.0292],
         [-0.0456,  0.2577, -0.1373,  ...,  0.0076,  0.0935,  0.02